#Imports

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter

#Parameters

In [ ]:
input_size = 28*28
output_size = 10
#batch_size = 32
learning_rate = 1e-3
#no_of_nodes = 128
epochs = 50
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))
loss_fn = nn.CrossEntropyLoss()

#Methods for training and testing

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def test(dataloader, model, epoch):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    writer.add_scalar("Avg loss", test_loss, epoch)
    writer.add_scalar("Accuracy", correct * 100, epoch)

In [ ]:
#Loads data sets

In [ ]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)



#Runs the training and test with diffrent parameters

In [ ]:
# Define model
for no_of_nodes in [64,128,256,512]:
  class NeuralNetwork(nn.Module):
      def __init__(self):
          super(NeuralNetwork, self).__init__()
          self.flatten = nn.Flatten()
          self.linear_relu_stack = nn.Sequential(
              nn.Linear(input_size, no_of_nodes),
              nn.ReLU(),
              nn.Linear(no_of_nodes, no_of_nodes),
              nn.ReLU(),
              nn.Linear(no_of_nodes, no_of_nodes),
              nn.ReLU(),
              nn.Linear(no_of_nodes, output_size),
              nn.ReLU()
          )

      def forward(self, x):
          x = self.flatten(x)
          logits = self.linear_relu_stack(x)
          return logits

  
  for batch_size in [32,64,128]:
    train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)
    model = NeuralNetwork().to(device)
    #creates a writer using params as the name of the run
    writer = SummaryWriter(comment=f'NN_2_hidden_layers__nodes_count_{no_of_nodes}__batch_size_{batch_size}__lr_{learning_rate}')
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    for t in range(epochs):
        print(f'Epoch {t+1} NN_2_hidden_layers__nodes_count_{no_of_nodes}__batch_size_{batch_size}__lr_{learning_rate}')
        train(train_dataloader, model, loss_fn, optimizer)
        test(test_dataloader, model, t+1)
    writer.flush()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs